<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, we have visualiz=sed the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this notebook, we will be performing more interactive visual analytics using `Folium`.


## Objectives



- Mark all launch sites on a map
- Mark the success/failed launches for each site on the map
- alculate the distances between a launch site to its proximities

Let's first import required Python packages:


In [1]:
import folium
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

## 1. Marking all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [3]:
import requests
import io

url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv"
response = requests.get(url)

spacex_df=pd.read_csv(io.StringIO(response.text))

with open("../data/spacex_launch_geo.csv", "w", encoding="utf-8") as file:
    file.write(response.text)

spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


Now, we can take a look at what are the coordinates for each site.


In [4]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualise those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [5]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We can use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate.


In [6]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

site_map

Now, let's add a circle for each launch site in data frame `launch_sites`


In [7]:
# Initial the map
site_map = folium.Map(location = [30, -100], zoom_start=4)

# Drawing the Equator line
folium.PolyLine([(0, -180), (0, 180)], color="green", weight=2.5, opacity=1).add_to(site_map)

for site, lat, lng in zip(launch_sites_df['Launch Site'], launch_sites_df['Lat'], launch_sites_df['Long']):
    site_coords = [lat, lng]

    # Create a circle for each launch site
    circle = folium.Circle(site_coords, radius=1000, color='#d35400', fill=True).add_child(folium.Popup(site))

    # Create a marker for each launch site with an icon showing its name
    marker = folium.map.Marker(
        site_coords,
        # Create an icon as a text label
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#D35400;"><b>%s</b></div>' % site,
            )
        )
    
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map

Findings:

* Proximity to the Equator:
None of the launch sites are directly on the Equator, but they are relatively closer to it, especially when compared to more polar regions. Being closer to the Equator provides a rotational speed advantage for launches, making it easier to reach the required speeds for orbit.

* Proximity to the Coast:
All of the launch sites are very close to the coast. 

In summary, while not directly on the Equator, the launch sites are strategically positioned in lower latitudes to take advantage of Earth's rotation. Their proximity to coastlines provides safety and logistical benefits for space launches.

# 2. Marking all success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [8]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [9]:
marker_cluster = MarkerCluster()


In [10]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    # If class=1, marker_color value will be green
    if launch_outcome == 1:
        return 'green'
    # If class=0, marker_color value will be red
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [11]:
# Initialize the map
site_map = folium.Map(location=[30, -100], zoom_start=4)

# Add marker_cluster to the map
site_map.add_child(marker_cluster)

# Iterate over spacex_df and add a marker for each launch to the MarkerCluster
for index, record in spacex_df.iterrows():
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color']),
        #popup=folium.Popup(record['Launch Site'], parse_html=True)  
    )
    marker_cluster.add_child(marker)

# Add filled circles for each launch site with popups displaying their names
# (Outside the MarkerCluster so they are always visible and not clustered)
for index, site in launch_sites_df.iterrows():
    site_coords = [site['Lat'], site['Long']]
    site_name = site['Launch Site']
    color = '#eb6e0e'

    folium.Circle(
        location=site_coords,
        radius=50,  # You can adjust the radius as needed
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.4,
        popup=folium.Popup(site_name, parse_html=True)
    ).add_to(site_map)


site_map

# 3. Distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while we are exploring the map, we can easily find the coordinates of any points of interests (such as railway)


In [12]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)

In [13]:
# Extract coordinates for "CCAFS SLC-40"
site_lat = launch_sites_df[launch_sites_df['Launch Site'] == 'CCAFS SLC-40']['Lat'].values[0]
site_lon = launch_sites_df[launch_sites_df['Launch Site'] == 'CCAFS SLC-40']['Long'].values[0]
site_coords = [site_lat, site_lon]

# Zoom coordinates
lon = -80.5769
lat = 28.56262
zoom_coords = [lat, lon]

# Initialize a zoomed-in map focused on "CCAFS SLC-40"
zoomed_map = folium.Map(location=zoom_coords, zoom_start=18)

# Add children from the original map to the new map
for child in site_map._children.values():
    zoomed_map.add_child(child)

# Add a filled circle marker for "CCAFS SLC-40" with a popup displaying its name
popup = folium.Popup('CCAFS SLC-40', parse_html=True, max_width=300)
circle = folium.Circle(
    location=site_coords,
    radius=50,
    color='#eb6e0e',
    fill=False,
    fill_opacity=0.4,
    popup=popup
).add_to(zoomed_map)

# Open the popup immediately
popup.show = True

zoomed_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [14]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

After marking down a point on the closest coastline using MousePosition and we will calculate the distance between the coastline point and the launch site.


In [15]:
# find coordinates of "CCAFS SLC-40" proximities
# Coastline coordinates
coastline_lat = site_lat
coastline_lon  = -80.56796

# Highway coordinates
highway_lat = (site_lat + 0.0001)
highway_lon = -80.57074

# Calculate distance between launch site and coastline
distance_coastline = calculate_distance(site_lat, site_lon, coastline_lat, coastline_lon)
distance_highway = calculate_distance(site_lat, site_lon, highway_lat, highway_lon)

In [16]:
# Interest points coordinates
coastline_coords = [coastline_lat, coastline_lon]
highway_coords = [highway_lat, highway_lon]


## Part 1: COASTLINE

# Display the distance between coastline point and launch site using the icon property 
cst_dst_marker = folium.Marker(
   coastline_coords,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
       ),
       popup="coastline"
   )

# Create a marker with 'x' at the coastline coordinates
x_marker = folium.Marker(
    coastline_coords,
    icon=folium.Icon(icon="times", prefix="fa", color='blue'),  # Using Font Awesome icon for 'x'
    popup=folium.Popup("coastline", parse_html=True)
)

# Coastline to launch site line coordinates
cst_to_site_coords = [(coastline_lat, coastline_lon), (site_lat, site_lon)]

# Add a PolyLine between coastline and launch site and display the distance on the line
cst_to_site_line = folium.PolyLine(
    locations=cst_to_site_coords,
    weight=1,
    color='blue'
).add_child(folium.Popup(f"Distance to site: {distance_coastline:.2f} KM"))

site_map.add_child(cst_dst_marker)
site_map.add_child(cst_to_site_line)
site_map.add_child(x_marker)


## Part 2: HIGHWAY

# Display the distance between Highway point and launch site using the icon property 
hw_dst_marker = folium.Marker(
   highway_coords,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway),
       )
   )

# Create a marker with 'x' at the Highway coordinates
x_marker_2 = folium.Marker(
    highway_coords,
    icon=folium.Icon(icon="times", prefix="fa", color='blue'),  # Using Font Awesome icon for 'x'
    popup=folium.Popup("highway", parse_html=True)
)

# Highway to launch site line coordinates
hw_to_site_coords = [(highway_lat, highway_lon), (site_lat, site_lon)]

# Add a PolyLine between Highway and launch site and display the distance on the line
hw_to_site_line = folium.PolyLine(
    locations=hw_to_site_coords,
    weight=1,
    color='#0eebeb'
).add_child(folium.Popup(f"Distance to site: {distance_highway:.2f} KM"))

site_map.add_child(hw_dst_marker)
site_map.add_child(hw_to_site_line)
site_map.add_child(x_marker_2)

site_map

In [17]:
# Create a new map with updated location and zoom level
coastline_map = folium.Map(location=[28.5632, -80.5725], zoom_start=17)

# Add children from the original map to the new map
for child in site_map._children.values():
    coastline_map.add_child(child)

coastline_map

In [18]:
# Coordinates between "CCAFS SLC-40" and  "Cape Canaveral" 
cc_lat = 28.40138
cc_lon = -80.60386
cc_coords = [cc_lat, cc_lon]

# Calculate distance between launch site and "Cape Canaveral" 
distance_cc = calculate_distance(site_lat, site_lon, cc_lat, cc_lon)

## Part 2: "Cape Canaveral" 

# Display the distance between "Cape Canaveral"  point and launch site using the icon property 
cc_dst_marker = folium.Marker(
   cc_coords,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_cc),
       )
   )

# Create a marker with 'x' at the "Cape Canaveral"  coordinates
x_marker_3 = folium.Marker(
    cc_coords,
    icon=folium.Icon(icon="times", prefix="fa", color='black'),  # Using Font Awesome icon for 'x'
    popup=folium.Popup("Cape Canaveral", parse_html=True)
)

# "Cape Canaveral"  to launch site line coordinates
cc_to_site_coords = [(cc_lat, cc_lon), (site_lat, site_lon)]

# Add a PolyLine between "Cape Canaveral"  and launch site and display the distance on the line
cc_to_site_line = folium.PolyLine(
    locations=cc_to_site_coords,
    weight=1,
    color='black'
).add_child(folium.Popup(f"Distance to site: {distance_cc:.2f} KM"))

site_map.add_child(cc_dst_marker)
site_map.add_child(cc_to_site_line)
site_map.add_child(x_marker_3)

In [19]:
# Create a new map with updated location and zoom level
interest_points_map = folium.Map(location=[28.47939, -80.57373], zoom_start=11)

# Add children from the original map to the new map
for child in site_map._children.values():
    interest_points_map.add_child(child)

interest_points_map

## Insights

#### Case: "CCAFS SLC-40" Launch Site

* **Proximity to Coast:**

  Upon closely examining the launch site at "CCAFS SLC-40," it's evident that it's strategically located very close to the coast, with a mere distance of 0.87KM separating them. Such proximity to coastlines is a standard characteristic of launch sites. The reasons are twofold:
  
  1. **Safety Concerns:** Launching over water ensures that if there's a mishap, the debris from the rocket falls harmlessly into the ocean. This minimizes risks to populated areas on land.
  
  2. **Orbital Flexibility:** Coastal launch sites offer flexibility in choosing launch azimuths, which is particularly beneficial when aiming for specific orbital inclinations.

* **Proximity to Highways:**

  Another noticeable trend is the site's closeness to highways. Being just 0.59KM away from the nearest highway is crucial for logistical reasons. Highways (and potentially other rapid transit routes like railways) play a pivotal role in the swift and efficient transportation of equipment, materials, and personnel.

* **Proximity to Cities:**

  On the other hand, while the site maintains proximity to coasts and highways, it is considerably distanced from populated cities. With Cape Canaveral being 18.2KM away, this deliberate distancing serves as a safety measure. It's a precaution to ensure that in the event of a malfunction, especially one that results in an explosion, the surrounding populace remains protected.

In summary, the positioning of launch sites like "CCAFS SLC-40" is a delicate balance between ensuring operational efficiency and maintaining public safety. The site's location reflects a well-thought-out strategy that prioritizes both logistics and safety.